# Data Preparation

This notebook processes raw product data from Excel, cleans specific fields, and prepares a JSONL file suitable for generating embeddings.

In [1]:
import pandas as pd
import json
import html
import re
import os

## 1. Load Data

In [5]:
# Define file paths
csv_output_path = 'D92-images.csv'
jsonl_output_path = 'product_embeddings.jsonl'

# Save as CSV (preserving original workflow requirement)
# index=False prevents pandas from writing a row number column
df = pd.read_csv(csv_output_path)

## 2. Data Cleaning & Transformation

We apply the following cleaning steps:
- **Product Name**: Decode HTML entities, fix encoding artifacts (smart quotes/dashes), and normalize whitespace.
- **Hierarchy Level 1**: Remove leading numeric codes (e.g., "92 GROCERY" -> "GROCERY").
- **Content Construction**: Combine hierarchy levels and product name into a single descriptive string.

In [ ]:
# Setup regex patterns
# Matches codes like OKA4, MFF30, IEF31, VTF5, GP27 (2-3 letters followed by 2 numbers)
noise_pattern = re.compile(r'\b[A-Z]{2,3}\d{2,3}\b')

def clean_text(text):
    if pd.isna(text):
        return ""
    text = str(text)
    
    # 1. Fix Encoding
    text = text.replace('â€™', "'").replace('â€“', "-").replace('â€”', "-")
    text = text.replace('\u00c3\u00b1', 'ñ').replace('Ã±', 'ñ')
    text = text.replace('\u00c2\u00ae', '')
    text = text.replace('\u00e2\u20ac\u201d', '-')

    # 2. Decode HTML
    text = html.unescape(text)

    # 3. Remove Inventory Codes
    text = noise_pattern.sub('', text)

    # 4. Normalize whitespace
    return " ".join(text.split())

def clean_level1(text):
    if pd.isna(text):
        return ""
    text = str(text)
    # Remove leading digits and whitespace
    return re.sub(r'^\d+\s*', '', text)

# Apply cleaning functions
df['clean_prod_nm'] = df['PROD_NM'].apply(clean_text)
df['clean_level1'] = df['RPT_HRCHY_LEVEL1_DESC'].apply(clean_level1)

# Ensure other fields are clean strings
df['clean_level0'] = df['RPT_HRCHY_LEVEL0_DESC'].fillna('').astype(str).str.strip()
df['clean_level2'] = df['RPT_HRCHY_LEVEL2_DESC'].fillna('').astype(str).str.strip()

# Create the formatted content string
df['content'] = (
    "Product: " + df['clean_prod_nm'] + ", " +
    "Level 0: " + df['clean_level0'] + ", " +
    "Level 1: " + df['clean_level1'] + ", " +
    "Level 2: " + df['clean_level2']
)

# Display a sample to verify cleaning
df[['PROD_NM', 'content']].head()

## 3. Export to JSONL

In [19]:
# Select the GTIN and content columns, and rename 'GTIN' to 'id'
output_df = df[['GTIN', 'content']].rename(columns={'GTIN': 'id'})

# Prepare the records for JSONL
records = output_df.to_dict(orient='records')

# Write to file
with open(jsonl_output_path, 'w', encoding='utf-8') as f:
    for record in records:
        f.write(json.dumps(record) + '\n')

print(f"Successfully wrote {len(records)} records to {jsonl_output_path}")

Successfully wrote 1000 records to product_embeddings.jsonl
